In [2]:
import torchtune
import os, sys
sys.path.insert(0, "..")
from recipes.r1_full_single_device import FullRLRecipeSingleDevice
from omegaconf import OmegaConf

import error: No module named 'triton'


In [3]:
cfg = OmegaConf.load("../recipes/configs/llama3_2/3B_full_rl_single_device.yaml")

In [4]:
recipe = FullRLRecipeSingleDevice(cfg)

INFO:torchtune.utils._logging:Hint: enable_activation_checkpointing is True, but enable_activation_offloading isn't. Enabling activation offloading should reduce memory further.
DEBUG:torchtune.utils._logging:Setting manual seed to local seed 1998871262. Local seed is seed + rank = 1998871262 + 0


In [5]:
recipe.setup(cfg=cfg)

Writing logs to /Users/ariel/checkpoint/tmp/full-llama3.2-finetune/log_1737933040.txt


INFO:torchtune.utils._logging:Model is initialized with precision torch.bfloat16.
INFO:torchtune.utils._logging:Tokenizer is initialized from file.
/Users/ariel/projects/torchtune-rl/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
INFO:torchtune.utils._logging:In-backward optimizers are set up.
INFO:torchtune.utils._logging:Loss is initialized.


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


INFO:torchtune.utils._logging:Dataset and Sampler are initialized.
INFO:torchtune.utils._logging:No learning rate scheduler configured. Using constant learning rate.
INFO:torchtune.utils._logging: Profiler config after instantiation: {'enabled': False}


In [6]:
for i, batch in enumerate(recipe._dataloader):
    print(i)
    print(batch)
    break

0
{'tokens': tensor([[128000,     32,  10652,   1990,   2724,    323,  22103,     13,    578,
           1217,  17501,    264,   3488,     11,    323,    279,  22103,  68577,
            433,     13,    578,  18328,   1176,  15849,    922,    279,  33811,
           1920,    304,    279,   4059,    323,   1243,   5825,    279,   1217,
            449,    279,   4320,     13,    578,  33811,   1920,    323,   4320,
            527,  44910,   2949,    366,  27963,   1500,  27963,     29,    323,
            366,   9399,   1500,   9399,     29,   9681,     11,  15947,     11,
            602,   1770,   2637,    366,  27963,     29,  20489,    287,   1920,
           1618,    524,  27963,     29,    366,   9399,     29,   9399,   1618,
            524,   9399,  14611,   2724,     25,  55357,   6944,    311,   3780,
            264,  13339,   4447,    430,   7194,    220,     17,  28518,   2734,
          14801,     13,  55357,    706,    220,   4370,   3117,  58174,     11,
            323

In [7]:
tokens = batch["tokens"].to("mps")

In [8]:
out_tokens, out_logits = recipe.generate_trajectory(tokens)

In [9]:
batch["answers"]

['7', '15', '16', '32']

In [10]:
recipe._tokenizer.encode("</answer>", False, False)

[524, 9399, 29]

In [53]:
recipe._tokenizer.encode("Something<|finetune_right_pad_id|>blah", True, True)

[128000,
 23958,
 27,
 91,
 5589,
 295,
 2957,
 10762,
 31390,
 851,
 91,
 29,
 71714,
 128001]

In [54]:
recipe._model.tok_emb

TransformerDecoder(
  (tok_embeddings): Embedding(128256, 3072)
  (layers): ModuleList(
    (0-27): 28 x CheckpointWrapper(
      (_checkpoint_wrapped_module): TransformerSelfAttentionLayer(
        (attn): MultiHeadAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (output_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (pos_embeddings): Llama3ScaledRoPE()
        )
        (mlp): FeedForward(
          (w1): Linear(in_features=3072, out_features=8192, bias=False)
          (w2): Linear(in_features=8192, out_features=3072, bias=False)
          (w3): Linear(in_features=3072, out_features=8192, bias=False)
          (activation): SiLU()
        )
        (sa_norm): RMSNorm()
        (mlp_norm): RMSNorm()
        (sa_scale): Identity()
        (mlp_scale): Identity()


In [35]:
recipe._tokenizer.decode(out_tokens[0].tolist(), False, False)

'<|begin_of_text|>A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think></think> and <answer></answer> tags, respectively, i.e., <think>reasoning process here</think> <answer>answer here</answer>. User: Gerald wants to buy a meat pie that costs 2 pfennigs. Gerald has 54 farthings, and there are 6 farthings to a pfennig. How many pfennigs will Gerald have left after buying the pie?. Assistant:<|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_right_pad_id|><|finetune_righ

In [45]:
out_tokens[0][150]

tensor(128004, device='mps:0')

In [49]:
out_logits[0, 152].argmax()

tensor(13, device='mps:0')

In [20]:
batch

{'tokens': tensor([[128000,     32,  10652,   1990,   2724,    323,  22103,     13,    578,
            1217,  17501,    264,   3488,     11,    323,    279,  22103,  68577,
             433,     13,    578,  18328,   1176,  15849,    922,    279,  33811,
            1920,    304,    279,   4059,    323,   1243,   5825,    279,   1217,
             449,    279,   4320,     13,    578,  33811,   1920,    323,   4320,
             527,  44910,   2949,    366,  27963,   1500,  27963,     29,    323,
             366,   9399,   1500,   9399,     29,   9681,     11,  15947,     11,
             602,   1770,   2637,    366,  27963,     29,  20489,    287,   1920,
            1618,    524,  27963,     29,    366,   9399,     29,   9399,   1618,
             524,   9399,  14611,   2724,     25,  55357,   6944,    311,   3780,
             264,  13339,   4447,    430,   7194,    220,     17,  28518,   2734,
           14801,     13,  55357,    706,    220,   4370,   3117,  58174,     11,
      

# Next TODO:
- [ ] Properly format the prompt in the data
- [ ] Do the RL training loop

RESEARCH IDEA:
Continuous RL for CoT? instead of predicting tokens, we predict the continuous embeddings